# Project 5: Generative AI Applications (P5)

Author: Christopher Aaron O'Hara

Dataset target: LANL authentication event stream (public CSR datasets)
- https://csr.lanl.gov/data/auth/
- https://csr.lanl.gov/data/cyber1/

Task: Transformer-based event sequence generation with RCA-oriented narrative interpretation.

This notebook is intentionally independent of P6. It produces structured generated artifacts that can be integrated later in P7.

## Motivation

IIoT and local-cloud operations teams often receive high-volume event streams but limited analyst time. A compact generative model can produce candidate event narratives for stress testing and analyst support, as long as generated outputs are clearly labeled, quality-checked, and never treated as ground truth.

This project focuses on a defensible generative workflow: train a Transformer language model on real LANL-style authentication event sequences, generate multiple samples, evaluate quality and failure modes, and derive RCA-style hypotheses from generated behavior patterns.

In [1]:
# Colab/Server GPU check (T4 expected)
import os
import subprocess
import torch

print("Python executable:", os.sys.executable)
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"GPU {i}: {props.name}")
        print(f"  Total VRAM (GB): {props.total_memory / (1024**3):.2f}")
        print(f"  Compute capability: {props.major}.{props.minor}")
else:
    print("No CUDA GPU visible to this kernel.")

print("\n=== nvidia-smi ===")
try:
    out = subprocess.check_output(["nvidia-smi"], stderr=subprocess.STDOUT, text=True)
    print(out[:2000])  # truncate for notebook readability
except Exception as e:
    print("nvidia-smi not available or failed:", e)


Python executable: /usr/bin/python3
Torch version: 2.10.0+cu128
CUDA available: True
CUDA device count: 1
GPU 0: Tesla T4
  Total VRAM (GB): 14.56
  Compute capability: 7.5

=== nvidia-smi ===
Fri Feb 20 05:35:08 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             11W /   7

## 1) Setup and Dependency Bootstrap

This cell installs missing core dependencies for a clean environment run.

In [42]:
import importlib.util
import subprocess
import sys

REQUIRED_PACKAGES = {
    'numpy': 'numpy',
    'pandas': 'pandas',
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'torch': 'torch',
    'networkx': 'networkx',
}


def ensure_packages(required):
    missing = []
    for import_name, pip_name in required.items():
        if importlib.util.find_spec(import_name) is None:
            missing.append(pip_name)

    status = {'missing_before_install': missing.copy(), 'installed_now': []}
    if missing:
        cmd = [sys.executable, '-m', 'pip', 'install', '--quiet'] + missing
        subprocess.check_call(cmd)
        status['installed_now'] = missing
    return status


bootstrap_status = ensure_packages(REQUIRED_PACKAGES)
print('Bootstrap status:', bootstrap_status)

Bootstrap status: {'missing_before_install': [], 'installed_now': []}


## 2) Imports and Reproducibility Controls

In [3]:
from pathlib import Path
import os
import re
import gzip
import json
import math
import random
import hashlib
from datetime import datetime, timezone
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print({'seed': SEED, 'device': DEVICE})

# GPU-oriented math setting (safe no-op on CPU or older torch)
try:
    torch.set_float32_matmul_precision('high')
except Exception:
    pass


{'seed': 42, 'device': 'cuda'}


## 3) Configuration

This section defines local data paths, processed-cache behavior, and training controls for the LANL 2017 network-flow workflow used in this notebook.


In [22]:
# Paths and run configuration (CPU/local)
DATASET_MODE = 'lanl2017_network'
DATA_DIR_OVERRIDE = os.environ.get('P5_DATA_DIR', '').strip()
EXTRA_RAW_SEARCH_DIRS = []

candidate_data_dirs = []
if DATA_DIR_OVERRIDE:
    candidate_data_dirs.append(Path(DATA_DIR_OVERRIDE))
candidate_data_dirs.extend([
    Path('data'),
    Path('P5/data'),
    Path('../P5/data'),
])

selected_data_dir = None
for cand in candidate_data_dirs:
    try:
        if cand.exists() and ((cand / 'raw').exists() or (cand / 'processed').exists()):
            selected_data_dir = cand
            break
    except Exception:
        continue

if selected_data_dir is None:
    selected_data_dir = Path('data')

DATA_DIR = selected_data_dir
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_BASE_DIR = DATA_DIR / 'processed'

PROCESSED_SUBDIR = os.environ.get('P5_PROCESSED_SUBDIR', '').strip()
if not PROCESSED_SUBDIR and (PROCESSED_BASE_DIR / '2017').exists():
    PROCESSED_SUBDIR = '2017'
PROCESSED_DIR = (PROCESSED_BASE_DIR / PROCESSED_SUBDIR) if PROCESSED_SUBDIR else PROCESSED_BASE_DIR
MODEL_DIR = PROCESSED_DIR / 'models'

RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_BASE_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Raw-first behavior for CPU/local workflow
USE_PROCESSED_CACHE_IF_AVAILABLE = False
WRITE_PROCESSED_CACHE = True
ENFORCE_RAW_MB_PER_SOURCE = True
MAX_RAW_MB_PER_SOURCE = 50

# Optional raw controls
_env_raw_list = os.environ.get('P5_RAW_FILE_LIST', '').strip()
RAW_FILE_LIST = [x.strip() for x in _env_raw_list.split(';') if x.strip()]
RAW_FILE_OVERRIDE = os.environ.get('P5_RAW_FILE_OVERRIDE', '').strip()
AUTO_SELECT_2017_NETFLOW = True
AUTO_SELECT_2017_MAX_DAYS = 2

# Optional WLS context controls
WLS_CONTEXT_FILE_LIST = []
AUTO_SELECT_WLS_CONTEXT = True
AUTO_SELECT_WLS_MAX_FILES = 2
MAX_WLS_CONTEXT_LINES = 50000

NETWORK2017_PATH = RAW_DIR / '2017' / 'netflow_day-03.csv'

# Parsing/sample controls
MAX_EVENTS = 120000
MIN_TOK_FREQ = 3
MAX_VOCAB = 20000
SEQ_LEN = 18
STRIDE = 6

# EDA controls
EDA_ENABLE_PLOTS = True
EDA_SAMPLE_MAX = 30000

# Tokenization controls
TIME_BIN_SECONDS = 300
COMP_BUCKETS = 2048
TRAIN_FRACTION = 0.90

# Model/training controls (CPU-safe defaults)
BATCH_SIZE = 128
EPOCHS = 10
EMBED_DIM = 128
NUM_LAYERS = 2
DROPOUT = 0.20
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4

D_MODEL = EMBED_DIM
N_HEADS = 4
N_LAYERS = NUM_LAYERS
BLOCK_SIZE = 64

GEN_MAX_NEW_TOKENS = 96
GEN_TEMPERATURE = 0.9
GEN_TOP_K = 20
NUM_GENERATIONS = 8

RUN_ABLATIONS = True
ABLATION_EPOCHS = 2
SAVE_MODEL_CHECKPOINTS = True
SAVE_ALL_ABLATION_CHECKPOINTS = False

DATALOADER_NUM_WORKERS = 0
DATALOADER_PIN_MEMORY = False
DATALOADER_PERSISTENT_WORKERS = False

# Optional runtime budget (None => uncapped)
MAX_TRAIN_MINUTES = None
TRAINING_BUDGET_MINUTES = MAX_TRAIN_MINUTES

PROCESSED_EVENTS_PATH = PROCESSED_DIR / f'p5_{DATASET_MODE}_events_subset.csv.gz'
PROCESSED_EVENTS_META_PATH = PROCESSED_DIR / f'p5_{DATASET_MODE}_events_meta.json'
PROCESSED_WLS_PATH = PROCESSED_DIR / 'p5_wls_context_subset.csv.gz'
TOKENIZER_PATH = PROCESSED_DIR / f'p5_{DATASET_MODE}_tokenizer.pkl'
MODEL_BASELINE_PATH = MODEL_DIR / 'baseline_lm.pt'
MODEL_ABLATION_PATH = MODEL_DIR / 'ablation_lm.pt'
METRICS_PATH = PROCESSED_DIR / 'metrics_summary.csv'
GENERATIONS_PATH = PROCESSED_DIR / 'generated_samples.csv'
RCA_PATH = PROCESSED_DIR / 'rca_hypotheses.csv'

print({'DATA_DIR': str(DATA_DIR), 'RAW_DIR': str(RAW_DIR), 'PROCESSED_DIR': str(PROCESSED_DIR), 'DEVICE': DEVICE})


In [5]:
# Disabled for CPU/local workflow. Use cells 8 and 14 only.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content exists: True
/content/drive exists: True
/content/drive/MyDrive exists: True


KeyboardInterrupt: 

## 4) Data Ingestion and Inspection

This section loads LANL 2017 network-flow data from local files, displays representative samples, and checks structure. It reads line-by-line so very large files can be processed without loading the entire file into memory.

If a processed subset already exists, the notebook can load it directly to avoid repeated heavy ingest work.


### File Integrity Probe

This probe checks a small partition of each selected CSV/JSON source before full ingest. It reads only limited lines/chunks to detect corruption or delimiter/format issues early.

In [33]:
PROBE_NETFLOW_LINES = 2000
PROBE_RANDOM_LINES = 8
PROBE_WLS_LINES = 2000


def _probe_select_netflow_files():
    if RAW_FILE_LIST:
        return [Path(x) for x in RAW_FILE_LIST if Path(x).exists()]
    p2017 = RAW_DIR / '2017'
    if p2017.exists():
        return sorted(p2017.glob('netflow_day-*.csv'))[:AUTO_SELECT_2017_MAX_DAYS]
    return []


def _probe_select_wls_files():
    if WLS_CONTEXT_FILE_LIST:
        return [Path(x) for x in WLS_CONTEXT_FILE_LIST if Path(x).exists()]
    p2017 = RAW_DIR / '2017'
    if p2017.exists():
        return sorted(p2017.glob('wls_day-*.json'))[:AUTO_SELECT_WLS_MAX_FILES]
    return []


def _reservoir_sample_lines(path: Path, k: int = 8, max_lines: int = 2000):
    sample = []
    seen = 0
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.rstrip('\n')
            if not line:
                continue
            seen += 1
            if len(sample) < k:
                sample.append(line)
            else:
                j = random.randint(1, seen)
                if j <= k:
                    sample[j - 1] = line
            if seen >= max_lines:
                break
    return sample, seen


probe_netflow_files = _probe_select_netflow_files()
probe_wls_files = _probe_select_wls_files()

print({'probe_netflow_files': [str(p) for p in probe_netflow_files], 'probe_wls_files': [str(p) for p in probe_wls_files]})

netflow_probe_rows = []
for fp in probe_netflow_files:
    file_mb = fp.stat().st_size / (1024 * 1024)
    chunk_ok = False
    chunk_rows = 0
    chunk_cols = None
    chunk_err = None

    try:
        chunk = pd.read_csv(fp, nrows=PROBE_NETFLOW_LINES, header=None, on_bad_lines='skip')
        chunk_ok = True
        chunk_rows = int(len(chunk))
        chunk_cols = int(chunk.shape[1])
    except Exception as e:
        chunk_err = str(e)

    sampled, seen = _reservoir_sample_lines(fp, k=PROBE_RANDOM_LINES, max_lines=PROBE_NETFLOW_LINES)
    field_counts = [len(x.split(',')) for x in sampled]

    netflow_probe_rows.append({
        'file': fp.name,
        'size_mb': round(file_mb, 2),
        'chunk_ok': chunk_ok,
        'chunk_rows': chunk_rows,
        'chunk_cols': chunk_cols,
        'sample_lines_seen': seen,
        'sample_min_fields': int(min(field_counts)) if field_counts else None,
        'sample_max_fields': int(max(field_counts)) if field_counts else None,
        'chunk_error': chunk_err,
    })

netflow_probe_df = pd.DataFrame(netflow_probe_rows)
if len(netflow_probe_df) > 0:
    display(netflow_probe_df)

wls_probe_rows = []
for fp in probe_wls_files:
    file_mb = fp.stat().st_size / (1024 * 1024)
    valid = 0
    invalid = 0
    seen = 0

    with open(fp, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.rstrip('\n')
            if not line:
                continue
            seen += 1
            try:
                obj = json.loads(line)
                if isinstance(obj, dict):
                    valid += 1
                else:
                    invalid += 1
            except Exception:
                invalid += 1
            if seen >= PROBE_WLS_LINES:
                break

    wls_probe_rows.append({
        'file': fp.name,
        'size_mb': round(file_mb, 2),
        'lines_checked': seen,
        'json_valid': valid,
        'json_invalid': invalid,
        'valid_ratio': (valid / seen) if seen > 0 else None,
    })

wls_probe_df = pd.DataFrame(wls_probe_rows)
if len(wls_probe_df) > 0:
    display(wls_probe_df)

if len(netflow_probe_df) > 0:
    failed_chunks = (~netflow_probe_df['chunk_ok']).sum()
    print({'netflow_files': len(netflow_probe_df), 'netflow_failed_chunks': int(failed_chunks)})
if len(wls_probe_df) > 0:
    low_valid = (wls_probe_df['valid_ratio'] < 0.8).sum()
    print({'wls_files': len(wls_probe_df), 'wls_low_json_valid_ratio': int(low_valid)})


{'probe_netflow_files': [], 'probe_wls_files': []}


### Probe Analysis

The probe confirms each selected source is readable and structurally consistent before full ingest. If chunk parsing fails, field counts drift heavily, or WLS JSON validity is low, fix the file set before training. Missing values in optional WLS preview fields can be acceptable when specific keys are absent for certain event types, but core parse fields (time/event id) should remain mostly populated.


In [41]:
def _infer_day_from_name(name: str):
    m = re.search(r'day[-_]?0*(\d+)', str(name).lower())
    if m:
        return int(m.group(1))
    return None


def _unique_existing_dirs(paths):
    unique = []
    seen = set()
    for p in paths:
        try:
            d = Path(p).expanduser()
        except Exception:
            continue
        if not d.exists() or not d.is_dir():
            continue
        try:
            key = str(d.resolve())
        except Exception:
            key = str(d)
        if key not in seen:
            seen.add(key)
            unique.append(d)
    return unique


def _build_search_roots():
    roots = [
        RAW_DIR / '2017',
        RAW_DIR,
        DATA_DIR,
        Path.cwd() / 'data' / 'raw' / '2017',
        Path.cwd() / 'data' / 'raw',
        Path.cwd() / 'data',
        Path.cwd() / 'P5' / 'data' / 'raw' / '2017',
        Path.cwd() / 'P5' / 'data' / 'raw',
        Path.cwd() / 'P5' / 'data',
    ]
    for extra in EXTRA_RAW_SEARCH_DIRS:
        roots.append(Path(extra))
    return _unique_existing_dirs(roots)


def _discover_files(search_roots, patterns):
    found = []
    for root in search_roots:
        for pat in patterns:
            try:
                found.extend([p for p in root.rglob(pat) if p.is_file()])
            except Exception:
                continue

    unique = []
    seen = set()
    for p in found:
        try:
            key = str(p.resolve())
        except Exception:
            key = str(p)
        if key not in seen:
            seen.add(key)
            unique.append(p)
    return unique


def _pick_by_day(files, max_days: int):
    if not files:
        return []
    by_day = {}
    unknown = []
    for p in files:
        day = _infer_day_from_name(p.name)
        if day is None:
            unknown.append(p)
        else:
            by_day.setdefault(day, []).append(p)

    picked = []
    for day in sorted(by_day.keys()):
        day_files = sorted(by_day[day], key=lambda x: x.stat().st_size if x.exists() else 0, reverse=True)
        picked.append(day_files[0])
        if len(picked) >= max_days:
            return picked

    if len(picked) < max_days and unknown:
        unknown_sorted = sorted(unknown, key=lambda x: x.stat().st_size if x.exists() else 0, reverse=True)
        for p in unknown_sorted:
            picked.append(p)
            if len(picked) >= max_days:
                break

    return picked


def _auto_select_2017_netflow_files(max_days: int = 2):
    net_dir = RAW_DIR / '2017'
    if not net_dir.exists():
        return []
    files = sorted(net_dir.glob('netflow_day-*.csv'))
    if not files:
        files = sorted(net_dir.glob('netflow_day-*.txt'))
    return _pick_by_day(files, max_days=max_days)


def _auto_select_wls_files(max_files: int = 2):
    wls_dir = RAW_DIR / '2017'
    if not wls_dir.exists():
        return []
    files = sorted(wls_dir.glob('wls_day-*.json'))
    return files[:max_files]


def ensure_raw_files():
    if RAW_FILE_LIST:
        selected = []
        missing = []
        for fp in RAW_FILE_LIST:
            p = Path(fp)
            if p.exists() and p.is_file():
                selected.append(p)
            else:
                missing.append(fp)
        if missing:
            raise FileNotFoundError(
                f'RAW_FILE_LIST entries not found: {missing}. '
                'Use paths like data/raw/2017/netflow_day-03.csv'
            )
        return selected

    if RAW_FILE_OVERRIDE:
        p = Path(RAW_FILE_OVERRIDE)
        if p.exists() and p.is_file():
            return [p]

    if AUTO_SELECT_2017_NETFLOW:
        auto_files = _auto_select_2017_netflow_files(max_days=AUTO_SELECT_2017_MAX_DAYS)
        if auto_files:
            print('Auto-selected LANL 2017 netflow files:', [str(p) for p in auto_files])
            return auto_files

    search_roots = _build_search_roots()
    discovered = _discover_files(search_roots, ['netflow_day-*.csv', 'netflow_day-*.txt'])
    if discovered:
        discovered_sorted = sorted(discovered, key=lambda x: (_infer_day_from_name(x.name) or 9999, str(x)))
        selected = _pick_by_day(discovered_sorted, max_days=AUTO_SELECT_2017_MAX_DAYS)
        print('Discovered LANL netflow files:', [str(p) for p in selected])
        return selected

    raise FileNotFoundError(
        'No local netflow files found. '
        'Place LANL files under data/raw/2017/ (for example data/raw/2017/netflow_day-03.csv) '
        'or set P5_DATA_DIR / P5_RAW_FILE_LIST / P5_RAW_FILE_OVERRIDE.'
    )


def _iter_lines(path: Path):
    if path.suffix == '.gz':
        with gzip.open(path, 'rt', encoding='utf-8', errors='ignore') as f:
            for line in f:
                yield line.rstrip('\n')
    else:
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                yield line.rstrip('\n')


def _to_int(x):
    try:
        return int(float(str(x).strip()))
    except Exception:
        return None


def _to_float(x):
    try:
        return float(str(x).strip())
    except Exception:
        return None


def parse_lanl2017_network(line: str):
    parts = [p.strip() for p in line.split(',')]
    if len(parts) < 11:
        return None

    t, dur, src_dev, dst_dev, proto, src_port, dst_port, src_pkts, dst_pkts, src_bytes, dst_bytes = parts[:11]
    t_int = _to_int(t)
    if t_int is None:
        return None

    return {
        'time': t_int,
        'src_user': 'UNK',
        'dst_user': 'UNK',
        'src_comp': src_dev,
        'dst_comp': dst_dev,
        'auth_type': f'PROTO_{proto}',
        'logon_type': f'SPORT_{src_port}',
        'orientation': 'NETFLOW',
        'status': 'FLOW',
        'duration': _to_float(dur),
        'protocol': str(proto),
        'src_port': src_port,
        'dst_port': dst_port,
        'src_packets': _to_int(src_pkts),
        'dst_packets': _to_int(dst_pkts),
        'src_bytes': _to_int(src_bytes),
        'dst_bytes': _to_int(dst_bytes),
    }


raw_load_mode = 'processed_cache' if (USE_PROCESSED_CACHE_IF_AVAILABLE and PROCESSED_EVENTS_PATH.exists()) else 'raw_stream'
raw_paths = []

if raw_load_mode == 'processed_cache':
    raw_df = pd.read_csv(PROCESSED_EVENTS_PATH)
    print('Loaded processed event subset cache:', PROCESSED_EVENTS_PATH)
else:
    raw_paths = ensure_raw_files()
    print('Using raw files:', [str(p) for p in raw_paths])

if not WLS_CONTEXT_FILE_LIST and AUTO_SELECT_WLS_CONTEXT:
    auto_wls = _auto_select_wls_files(max_files=AUTO_SELECT_WLS_MAX_FILES)
    WLS_CONTEXT_FILE_LIST = [str(p) for p in auto_wls]
    if WLS_CONTEXT_FILE_LIST:
        print('Auto-selected WLS context files:', WLS_CONTEXT_FILE_LIST)

if raw_load_mode != 'processed_cache':
    byte_cap = int(MAX_RAW_MB_PER_SOURCE * 1024 * 1024)
    source_bytes = {}
    records = []

    for raw_file in raw_paths:
        day_num = _infer_day_from_name(raw_file.name)
        source_bytes.setdefault(raw_file.name, 0)

        for line in _iter_lines(raw_file):
            if ENFORCE_RAW_MB_PER_SOURCE and source_bytes[raw_file.name] >= byte_cap:
                break
            source_bytes[raw_file.name] += len(line.encode('utf-8', errors='ignore')) + 1

            obj = parse_lanl2017_network(line)
            if obj is not None:
                obj['source_file'] = raw_file.name
                obj['data_day'] = day_num
                records.append(obj)

            if MAX_EVENTS is not None and len(records) >= MAX_EVENTS:
                break

        if MAX_EVENTS is not None and len(records) >= MAX_EVENTS:
            break

    if len(records) < 2000:
        raise ValueError('Too few parsed events. Verify netflow file format.')

    raw_df = pd.DataFrame(records)

    if WRITE_PROCESSED_CACHE:
        raw_df.to_csv(PROCESSED_EVENTS_PATH, index=False, compression='gzip')
        meta = {
            'dataset_mode': DATASET_MODE,
            'source_files': [str(p) for p in raw_paths],
            'events': int(len(raw_df)),
            'max_events': int(MAX_EVENTS) if MAX_EVENTS is not None else None,
            'enforce_raw_mb_per_source': bool(ENFORCE_RAW_MB_PER_SOURCE),
            'max_raw_mb_per_source': float(MAX_RAW_MB_PER_SOURCE),
            'raw_bytes_read_per_source': source_bytes,
            'created_utc': datetime.now(timezone.utc).isoformat(),
        }
        with open(PROCESSED_EVENTS_META_PATH, 'w', encoding='utf-8') as f:
            json.dump(meta, f, indent=2)
        print('Wrote processed event subset cache:', PROCESSED_EVENTS_PATH)
        print('Wrote processed event metadata:', PROCESSED_EVENTS_META_PATH)

print('raw_load_mode:', raw_load_mode)
print('Loaded events:', len(raw_df))
print('Loaded source files:', raw_df['source_file'].nunique())
if 'data_day' in raw_df.columns:
    print('Loaded days:', sorted([d for d in raw_df['data_day'].dropna().unique().tolist()]))

display(raw_df.head(10))
display(raw_df.describe(include='all').head(12))


def _extract_wls_fields(line: str):
    try:
        obj = json.loads(line)
        if isinstance(obj, dict):
            return {
                'timestamp': obj.get('Time') or obj.get('time') or obj.get('TimeCreated') or obj.get('timestamp'),
                'event_id': obj.get('EventID') or obj.get('event_id') or obj.get('id'),
                'computer': obj.get('LogHost') or obj.get('Computer') or obj.get('computer') or obj.get('host'),
                'channel': obj.get('LogonTypeDescription') or obj.get('LogonType') or obj.get('Channel') or obj.get('channel'),
                'provider': obj.get('AuthenticationPackage') or obj.get('ParentProcessName') or obj.get('ProviderName') or obj.get('provider') or obj.get('Provider'),
                'user': obj.get('UserName') or obj.get('user'),
                'domain': obj.get('DomainName') or obj.get('domain'),
                'source': obj.get('Source') or obj.get('source'),
                'raw_line': line[:500],
            }
    except Exception:
        pass
    return {
        'timestamp': None,
        'event_id': None,
        'computer': None,
        'channel': None,
        'provider': None,
        'user': None,
        'domain': None,
        'source': None,
        'raw_line': line[:500],
    }


wls_context_df = None
if WLS_CONTEXT_FILE_LIST:
    preview_rows = []
    lines_seen = 0
    for fp in WLS_CONTEXT_FILE_LIST:
        p = Path(fp)
        if not p.exists():
            print('WLS context file not found:', p)
            continue
        for line in _iter_lines(p):
            row = _extract_wls_fields(line)
            row['source_file'] = p.name
            preview_rows.append(row)
            lines_seen += 1
            if lines_seen >= MAX_WLS_CONTEXT_LINES:
                break
        if lines_seen >= MAX_WLS_CONTEXT_LINES:
            break

    if preview_rows:
        wls_context_df = pd.DataFrame(preview_rows)
        print('Loaded WLS context lines:', len(wls_context_df))
        display(wls_context_df.head(10))

        if WRITE_PROCESSED_CACHE:
            wls_context_df.to_csv(PROCESSED_WLS_PATH, index=False, compression='gzip')
            print('Wrote processed WLS context subset:', PROCESSED_WLS_PATH)


Loaded processed event subset cache: /content/drive/MyDrive/data processed 2017/p5_lanl2017_network_events_subset.csv.gz


KeyboardInterrupt: 

In [35]:
# Disabled for CPU/local workflow. Use cells 8 and 14 only.

Loaded processed event subset cache: /content/drive/MyDrive/data processed 2017/p5_lanl2017_network_events_subset.csv.gz
Rows: 300000


,time,src_user,dst_user,src_comp,dst_comp,auth_type,logon_type,orientation,status,duration,protocol,src_port,dst_port,src_packets,dst_packets,src_bytes,dst_bytes,source_file,data_day
0,118781,UNK,UNK,Comp364445,Comp547245,PROTO_17,SPORT_Port05507,NETFLOW,FLOW,5580.0,17,Port05507,Port46272,0,755065,0,1042329018,netflow_day-02.csv,2
1,118783,UNK,UNK,Comp450942,Comp829338,PROTO_6,SPORT_Port03137,NETFLOW,FLOW,6976.0,6,Port03137,445,1665,1108,300810,250408,netflow_day-02.csv,2
2,118785,UNK,UNK,IP564116,Comp141988,PROTO_17,SPORT_5060,NETFLOW,FLOW,14178.0,17,5060,5060,1866,0,1477041,0,netflow_day-02.csv,2
3,118785,UNK,UNK,IP564116,Comp141988,PROTO_17,SPORT_5060,NETFLOW,FLOW,28147.0,17,5060,5060,3326,0,2656305,0,netflow_day-02.csv,2
4,118785,UNK,UNK,IP564116,Comp141988,PROTO_17,SPORT_5060,NETFLOW,FLOW,48507.0,17,5060,5060,5423,0,4388449,0,netflow_day-02.csv,2


In [ ]:
# Disabled for CPU/local workflow. Use cells 8 and 14 only.

Mounted at /content/drive


In [37]:
# Disabled for CPU/local workflow. Use cells 8 and 14 only.

raw_load_mode: processed_cache
Loaded events: 300000


,time,src_user,dst_user,src_comp,dst_comp,auth_type,logon_type,orientation,status,duration,protocol,src_port,dst_port,src_packets,dst_packets,src_bytes,dst_bytes,source_file,data_day
0,118781,UNK,UNK,Comp364445,Comp547245,PROTO_17,SPORT_Port05507,NETFLOW,FLOW,5580.0,17,Port05507,Port46272,0,755065,0,1042329018,netflow_day-02.csv,2
1,118783,UNK,UNK,Comp450942,Comp829338,PROTO_6,SPORT_Port03137,NETFLOW,FLOW,6976.0,6,Port03137,445,1665,1108,300810,250408,netflow_day-02.csv,2
2,118785,UNK,UNK,IP564116,Comp141988,PROTO_17,SPORT_5060,NETFLOW,FLOW,14178.0,17,5060,5060,1866,0,1477041,0,netflow_day-02.csv,2
3,118785,UNK,UNK,IP564116,Comp141988,PROTO_17,SPORT_5060,NETFLOW,FLOW,28147.0,17,5060,5060,3326,0,2656305,0,netflow_day-02.csv,2
4,118785,UNK,UNK,IP564116,Comp141988,PROTO_17,SPORT_5060,NETFLOW,FLOW,48507.0,17,5060,5060,5423,0,4388449,0,netflow_day-02.csv,2


In [38]:
# Disabled for CPU/local workflow. Use cells 8 and 14 only.

device: cuda
gpu: Tesla T4
Overrides set: MAX_EVENTS, BATCH_SIZE, EPOCHS, MAX_TRAIN_MINUTES


### Analysis

The ingestion output now reports whether data came from raw stream files or a processed cache, so reruns are transparent and fast. Per-source size caps create deterministic, shareable subsets while preserving real LANL structure. This supports reproducibility and reviewer portability without shipping multi-GB raw files.


## 5) Exploratory Data Analysis (EDA)

Before model training, this section quantifies source balance, temporal coverage, and numeric distribution shape. These checks help justify token design, generation settings, and ablation priorities.

In [39]:
eda_df = raw_df.copy()

print({'rows': len(eda_df), 'columns': list(eda_df.columns)[:12]})

if 'source_file' in eda_df.columns:
    src_counts = eda_df['source_file'].value_counts().reset_index()
    src_counts.columns = ['source_file', 'rows']
    display(src_counts)

if 'data_day' in eda_df.columns:
    day_counts = eda_df['data_day'].value_counts(dropna=False).sort_index().reset_index()
    day_counts.columns = ['data_day', 'rows']
    display(day_counts)

numeric_cols = [c for c in ['duration', 'src_packets', 'dst_packets', 'src_bytes', 'dst_bytes'] if c in eda_df.columns]
if numeric_cols:
    nan_summary = eda_df[numeric_cols].isna().sum().reset_index()
    nan_summary.columns = ['feature', 'nan_count']
    nan_summary['nan_pct'] = 100.0 * nan_summary['nan_count'] / max(len(eda_df), 1)
    nan_summary['nan_pct'] = nan_summary['nan_pct'].round(4)
    display(nan_summary)

    num_stats = eda_df[numeric_cols].describe().T
    num_stats['variance'] = eda_df[numeric_cols].var(numeric_only=True)
    num_stats['skewness'] = eda_df[numeric_cols].skew(numeric_only=True)
    num_stats['kurtosis'] = eda_df[numeric_cols].kurt(numeric_only=True)
    display(num_stats.round(4))

    cov_df = eda_df[numeric_cols].cov()
    display(cov_df.round(4))

    sample_n = min(EDA_SAMPLE_MAX, len(eda_df))
    plot_df = eda_df[numeric_cols].sample(n=sample_n, random_state=SEED) if sample_n < len(eda_df) else eda_df[numeric_cols]

    if EDA_ENABLE_PLOTS:
        fig, axes = plt.subplots(2, max(1, len(numeric_cols)), figsize=(4 * max(1, len(numeric_cols)), 7))
        axes = np.array(axes).reshape(2, -1)

        for j, col in enumerate(numeric_cols):
            sns.histplot(plot_df[col].dropna(), bins=50, ax=axes[0, j], color='#4C72B0')
            axes[0, j].set_title(f'{col} Histogram')
            axes[0, j].set_xlabel(col)
            axes[0, j].set_ylabel('Count')

            sns.boxplot(x=np.log1p(plot_df[col].dropna()), ax=axes[1, j], color='#55A868')
            axes[1, j].set_title(f'log1p({col}) Boxplot')
            axes[1, j].set_xlabel(f'log1p({col})')

        plt.tight_layout()
        plt.show()

        # Scale covariance for readable annotations (fixed decimals, shorter numbers)
        cov_abs_max = float(np.nanmax(np.abs(cov_df.to_numpy()))) if cov_df.size > 0 else 0.0
        if np.isfinite(cov_abs_max) and cov_abs_max > 0:
            cov_exp = int(np.floor(np.log10(cov_abs_max)))
            cov_exp3 = int(np.floor(cov_exp / 3.0) * 3)
            cov_scale = 10.0 ** cov_exp3
        else:
            cov_exp3 = 0
            cov_scale = 1.0

        cov_plot = cov_df / cov_scale

        plt.figure(figsize=(6, 5))
        sns.heatmap(cov_plot, annot=True, fmt='.2f', cmap='Blues', annot_kws={'size': 9})
        if cov_exp3 != 0:
            plt.title(f'Covariance Matrix (Numeric Event Fields, scaled by 1e{cov_exp3})')
        else:
            plt.title('Covariance Matrix (Numeric Event Fields)')
        plt.tight_layout()
        plt.show()

if 'protocol' in eda_df.columns:
    top_proto = eda_df['protocol'].astype(str).value_counts().head(12).reset_index()
    top_proto.columns = ['protocol', 'count']
    display(top_proto)

    if EDA_ENABLE_PLOTS:
        plt.figure(figsize=(8, 4))
        sns.barplot(data=top_proto, x='protocol', y='count', color='#C44E52')
        plt.title('Top Protocol Values')
        plt.xlabel('Protocol')
        plt.ylabel('Count')
        plt.tight_layout()
        plt.show()


{'rows': 300000, 'columns': ['time', 'src_user', 'dst_user', 'src_comp', 'dst_comp', 'auth_type', 'logon_type', 'orientation', 'status', 'duration', 'protocol', 'src_port']}


,source_file,rows
0,netflow_day-02.csv,300000


,data_day,rows
0,2,300000


,feature,nan_count,nan_pct
0,duration,0,0.0
1,src_packets,0,0.0
2,dst_packets,0,0.0
3,src_bytes,0,0.0
4,dst_bytes,0,0.0


,count,mean,std,min,25%,50%,75%,max,variance,skewness,kurtosis
duration,300000.0,1.379577e+06,1.792855e+06,0.0,100600.0,635432.0,1908822.50,7.655151e+06,3.214329e+12,1.6768,1.9940
src_packets,300000.0,7.867200e+05,3.036942e+07,0.0,2665.0,20018.0,66885.25,2.396174e+09,9.223015e+14,53.6403,3149.5173
dst_packets,300000.0,7.478724e+05,3.601702e+07,0.0,0.0,0.0,414.00,3.201805e+09,1.297226e+15,63.2702,4400.4956
src_bytes,300000.0,1.964466e+08,5.689635e+09,0.0,413364.0,4962866.0,39867216.00,4.193353e+11,3.237195e+19,50.6767,2799.9846
dst_bytes,300000.0,1.627924e+08,6.521489e+09,0.0,0.0,0.0,25806.00,5.434128e+11,4.252982e+19,55.7071,3511.0333


,duration,src_packets,dst_packets,src_bytes,dst_bytes
duration,3.214329e+12,-5.934940e+10,-2.853612e+11,1.131009e+14,-7.847991e+13
src_packets,-5.934940e+10,9.223015e+14,1.015642e+15,1.707465e+17,1.705096e+17
dst_packets,-2.853612e+11,1.015642e+15,1.297226e+15,1.808065e+17,2.200182e+17
src_bytes,1.131009e+14,1.707465e+17,1.808065e+17,3.237195e+19,2.999110e+19
dst_bytes,-7.847991e+13,1.705096e+17,2.200182e+17,2.999110e+19,4.252982e+19


NameError: name 'EDA_SAMPLE_MAX' is not defined

### Analysis

This EDA quantifies whether the subset preserves meaningful structure from the raw stream. Numeric moments (mean, variance, skewness, kurtosis), covariance, and protocol concentration provide evidence about heavy tails and imbalance, which informs tokenization and interpretation of generated sequences. Any NaNs in the numeric summary are expected only when fields are missing or non-parsable in sampled lines; persistent high NaN percentages in core numeric fields would indicate ingest quality issues that should be fixed before training.


## 6) Preprocessing and Sequence Construction

Events are transformed into compact symbolic tokens for sequence modeling. This section now enforces a modeling row cap (`MAX_EVENTS`) even when a larger processed cache is loaded, so quick and bounded runs stay consistent with runtime presets.

In [40]:
def _row_get(row, key, default='UNK'):
    # supports dict-like rows and namedtuples from itertuples
    if isinstance(row, dict):
        return row.get(key, default)
    return getattr(row, key, default)


def stable_bucket(value: str, n_buckets: int) -> int:
    if value is None:
        return 0
    s = str(value)
    h = hashlib.md5(s.encode('utf-8')).hexdigest()
    return int(h, 16) % n_buckets


def clean_token(s: str, default='UNK'):
    if s is None:
        return default
    s = str(s).strip()
    if s in {'', '?', 'nan', 'None'}:
        return default
    s = re.sub(r'[^A-Za-z0-9_-]', '_', s)
    return s[:40]


def magnitude_bin(v):
    try:
        x = max(float(v), 0.0)
        return int(np.log1p(x))
    except Exception:
        return 0


def event_to_tokens(row):
    t = _row_get(row, 'time', 0)
    t = t if pd.notna(t) else 0
    tbin = int(t) // TIME_BIN_SECONDS

    sc = stable_bucket(clean_token(_row_get(row, 'src_comp', 'UNK')), COMP_BUCKETS)
    dc = stable_bucket(clean_token(_row_get(row, 'dst_comp', 'UNK')), COMP_BUCKETS)

    proto = clean_token(_row_get(row, 'protocol', 'UNK'))
    sport = clean_token(_row_get(row, 'src_port', 'UNK'))
    dport = clean_token(_row_get(row, 'dst_port', 'UNK'))

    dur_b = magnitude_bin(_row_get(row, 'duration', 0))
    spk_b = magnitude_bin(_row_get(row, 'src_packets', 0))
    dpk_b = magnitude_bin(_row_get(row, 'dst_packets', 0))
    sby_b = magnitude_bin(_row_get(row, 'src_bytes', 0))
    dby_b = magnitude_bin(_row_get(row, 'dst_bytes', 0))

    return [
        f'TB_{tbin}',
        f'SC_{sc}',
        f'DC_{dc}',
        f'PR_{proto}',
        f'SP_{sport}',
        f'DP_{dport}',
        f'DU_{dur_b}',
        f'SPK_{spk_b}',
        f'DPK_{dpk_b}',
        f'SBY_{sby_b}',
        f'DBY_{dby_b}',
        '<EOS>',
    ]


model_df = raw_df.copy()
if MAX_EVENTS is not None and len(model_df) > int(MAX_EVENTS):
    if 'time' in model_df.columns:
        model_df = model_df.sort_values('time', kind='stable').head(int(MAX_EVENTS)).reset_index(drop=True)
    else:
        model_df = model_df.head(int(MAX_EVENTS)).reset_index(drop=True)

if len(model_df) < 2000:
    raise ValueError('Too few events for sequence modeling after cap/filter. Increase MAX_EVENTS.')

try:
    from tqdm.auto import tqdm as _tqdm
except Exception:
    _tqdm = None

rows_iter = model_df.itertuples(index=False, name='EventRow')
if _tqdm is not None:
    rows_iter = _tqdm(rows_iter, total=len(model_df), desc='Tokenizing events', leave=False)

all_token_sequences = [event_to_tokens(row) for row in rows_iter]
flat_tokens = [tok for seq in all_token_sequences for tok in seq]

counter = Counter(flat_tokens)
itos = ['<PAD>', '<UNK>'] + sorted(counter.keys())
stoi = {tok: i for i, tok in enumerate(itos)}
encoded = np.array([stoi.get(t, 1) for t in flat_tokens], dtype=np.int64)

split_idx = int(len(encoded) * TRAIN_FRACTION)
train_tokens = encoded[:split_idx]
val_tokens = encoded[split_idx:]

avg_tokens_per_event = len(flat_tokens) / max(len(all_token_sequences), 1)

print({
    'raw_rows_loaded': len(raw_df),
    'model_rows_used': len(model_df),
    'events': len(all_token_sequences),
    'total_tokens': len(flat_tokens),
    'avg_tokens_per_event': round(avg_tokens_per_event, 3),
    'vocab_size': len(itos),
    'train_tokens': len(train_tokens),
    'val_tokens': len(val_tokens),
})
print('Top 20 tokens:', counter.most_common(20))


Tokenizing events:   0%|          | 0/120000 [00:00<?, ?it/s]

NameError: name 'TIME_BIN_SECONDS' is not defined

### Analysis

Preprocessing now reports both `raw_rows_loaded` and `model_rows_used`, making it explicit when runtime caps are applied for bounded training. This keeps sequence construction consistent with quick and time-budgeted runs while preserving deterministic tokenization and split behavior.

## 7) Language-Model Dataset, Model Definition, and Training Utilities

In [ ]:
class TokenBlockDataset(Dataset):
    def __init__(self, token_array: np.ndarray, block_size: int):
        self.token_array = token_array
        self.block_size = block_size

    def __len__(self):
        return max(0, len(self.token_array) - self.block_size - 1)

    def __getitem__(self, idx):
        x = torch.tensor(self.token_array[idx: idx + self.block_size], dtype=torch.long)
        y = torch.tensor(self.token_array[idx + 1: idx + self.block_size + 1], dtype=torch.long)
        return x, y


train_ds = TokenBlockDataset(train_tokens, BLOCK_SIZE)
val_ds = TokenBlockDataset(val_tokens, BLOCK_SIZE)

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=DATALOADER_NUM_WORKERS,
    pin_memory=DATALOADER_PIN_MEMORY,
    persistent_workers=DATALOADER_PERSISTENT_WORKERS,
)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=True,
    num_workers=DATALOADER_NUM_WORKERS,
    pin_memory=DATALOADER_PIN_MEMORY,
    persistent_workers=DATALOADER_PERSISTENT_WORKERS,
)


class CausalTransformerLM(nn.Module):
    def __init__(self, vocab_size, d_model=128, n_heads=4, n_layers=3, dropout=0.2, max_len=512):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Parameter(torch.zeros(1, max_len, d_model))

        layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=4 * d_model,
            dropout=dropout,
            batch_first=True,
            activation='gelu',
        )
        self.encoder = nn.TransformerEncoder(layer, num_layers=n_layers)
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)

    def _causal_mask(self, T, device):
        mask = torch.triu(torch.ones(T, T, device=device), diagonal=1)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask

    def forward(self, x):
        B, T = x.shape
        tok = self.token_emb(x)
        pos = self.pos_emb[:, :T, :]
        h = tok + pos
        mask = self._causal_mask(T, x.device)
        h = self.encoder(h, mask=mask)
        h = self.norm(h)
        logits = self.head(h)
        return logits


def build_model(cfg):
    return CausalTransformerLM(
        vocab_size=len(itos),
        d_model=cfg['d_model'],
        n_heads=cfg['n_heads'],
        n_layers=cfg['n_layers'],
        dropout=cfg['dropout'],
        max_len=max(512, BLOCK_SIZE + GEN_MAX_NEW_TOKENS + 4),
    ).to(DEVICE)


def make_optimizer(model, cfg):
    return torch.optim.AdamW(model.parameters(), lr=cfg['lr'], weight_decay=cfg['weight_decay'])


criterion = nn.CrossEntropyLoss()
_base_cfg = {
    'd_model': D_MODEL,
    'n_heads': N_HEADS,
    'n_layers': N_LAYERS,
    'dropout': DROPOUT,
    'lr': LEARNING_RATE,
    'weight_decay': WEIGHT_DECAY,
}
_param_estimate = sum(p.numel() for p in build_model(_base_cfg).parameters())
print({'train_batches': len(train_loader), 'val_batches': len(val_loader), 'baseline_model_params_estimate': _param_estimate})


The architecture is a causal Transformer language model over event tokens. It is suitable for sequence generation because next-token prediction captures temporal and structural dependencies between event fields.

## 8) Train and Compare Generative Model Ablations

This section trains multiple controlled configurations, compares validation loss and perplexity, and selects the final model with a deterministic score rule. Checkpoints and tokenizer artifacts are saved for reproducible reruns without retraining.


In [ ]:
import time

try:
    from tqdm.auto import tqdm  # optional; falls back to plain iterator if unavailable
except Exception:
    tqdm = None


def _iter_with_progress(loader, desc, enabled=True, total=None):
    if enabled and tqdm is not None:
        kwargs = {'desc': desc, 'leave': False}
        if total is not None:
            kwargs['total'] = int(total)
        return tqdm(loader, **kwargs)
    return loader


def run_epoch(model, loader, optimizer=None, desc='epoch', max_batches=None, use_tqdm=True):
    is_train = optimizer is not None
    model.train(is_train)

    losses = []
    total_batches = len(loader)
    if total_batches == 0:
        raise ValueError(f'Loader has zero batches for {desc}. Reduce BLOCK_SIZE/BATCH_SIZE or increase token count.')

    effective_total = total_batches if max_batches is None else min(total_batches, int(max_batches))
    start_ts = time.time()
    iterator = _iter_with_progress(loader, desc=desc, enabled=use_tqdm, total=effective_total)

    for batch_idx, (x, y) in enumerate(iterator, start=1):
        x = x.to(DEVICE, non_blocking=DATALOADER_PIN_MEMORY)
        y = y.to(DEVICE, non_blocking=DATALOADER_PIN_MEMORY)
        logits = model(x)
        loss = criterion(logits.reshape(-1, logits.size(-1)), y.reshape(-1))

        if not torch.isfinite(loss):
            raise FloatingPointError(f'Non-finite loss detected in {desc} at batch {batch_idx}: {float(loss.detach().cpu())}')

        if is_train:
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        losses.append(float(loss.item()))

        if tqdm is None and (batch_idx == 1 or batch_idx % 25 == 0 or batch_idx == effective_total):
            elapsed = time.time() - start_ts
            it_per_sec = batch_idx / max(elapsed, 1e-9)
            eta = (effective_total - batch_idx) / max(it_per_sec, 1e-9)
            bar_w = 24
            done = int(bar_w * batch_idx / max(effective_total, 1))
            bar = '[' + '#' * done + '-' * (bar_w - done) + ']'
            print(
                f'{desc} {bar} {batch_idx}/{effective_total} '
                f'loss={np.mean(losses):.4f} it/s={it_per_sec:.2f} eta={eta:.1f}s'
            )

        if max_batches is not None and batch_idx >= max_batches:
            break

    return float(np.mean(losses)) if losses else np.nan


# Runtime knobs (can be overridden in config cell)
USE_TQDM_PROGRESS = globals().get('USE_TQDM_PROGRESS', True)
MAX_TRAIN_BATCHES_PER_EPOCH = globals().get('MAX_TRAIN_BATCHES_PER_EPOCH', None)
MAX_VAL_BATCHES_PER_EPOCH = globals().get('MAX_VAL_BATCHES_PER_EPOCH', None)
TRAINING_BUDGET_MINUTES = globals().get('TRAINING_BUDGET_MINUTES', None)
if TRAINING_BUDGET_MINUTES in [None, 0]:
    TRAINING_BUDGET_SECONDS = None
else:
    TRAINING_BUDGET_SECONDS = float(TRAINING_BUDGET_MINUTES) * 60.0
TRAIN_GLOBAL_START = time.time()

baseline_cfg = {
    'd_model': D_MODEL,
    'n_heads': N_HEADS,
    'n_layers': N_LAYERS,
    'dropout': DROPOUT,
    'lr': LEARNING_RATE,
    'weight_decay': WEIGHT_DECAY,
    'epochs': EPOCHS,
}
experiments = [{'name': 'baseline', **baseline_cfg}]

if RUN_ABLATIONS:
    experiments.extend([
        {'name': 'dropout_0p1', **{**baseline_cfg, 'dropout': 0.1, 'epochs': ABLATION_EPOCHS}},
        {'name': 'wider_d192_l4', **{**baseline_cfg, 'd_model': 192, 'n_heads': 6, 'n_layers': 4, 'epochs': ABLATION_EPOCHS}},
        {'name': 'low_lr_1e4', **{**baseline_cfg, 'lr': 1e-4, 'epochs': ABLATION_EPOCHS}},
    ])

run_store = {}
experiment_rows = []
training_run_id = f'p5_train_seed{SEED}_{datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")}'

print({
    'device': DEVICE,
    'experiments': [e['name'] for e in experiments],
    'train_batches_per_epoch': len(train_loader),
    'val_batches_per_epoch': len(val_loader),
    'max_train_batches_per_epoch': MAX_TRAIN_BATCHES_PER_EPOCH,
    'max_val_batches_per_epoch': MAX_VAL_BATCHES_PER_EPOCH,
    'training_budget_minutes': TRAINING_BUDGET_MINUTES,
    'use_tqdm_progress': USE_TQDM_PROGRESS,
})

for exp in experiments:
    if TRAINING_BUDGET_SECONDS is not None and (time.time() - TRAIN_GLOBAL_START) >= TRAINING_BUDGET_SECONDS:
        print('Time budget reached before starting next experiment. Stopping training loop.')
        break

    name = exp['name']
    print(f'\n=== Training experiment: {name} ===')

    model_i = build_model(exp)
    optimizer_i = make_optimizer(model_i, exp)
    history = {'epoch': [], 'train_loss': [], 'val_loss': []}
    best_val = math.inf
    best_state = None

    exp_start = time.time()
    for epoch in range(1, int(exp['epochs']) + 1):
        if TRAINING_BUDGET_SECONDS is not None and (time.time() - TRAIN_GLOBAL_START) >= TRAINING_BUDGET_SECONDS:
            print(f'[{name}] time budget reached before epoch {epoch}. Ending this experiment early.')
            break

        epoch_start = time.time()
        train_loss = run_epoch(
            model_i,
            train_loader,
            optimizer=optimizer_i,
            desc=f'{name} train e{epoch}',
            max_batches=MAX_TRAIN_BATCHES_PER_EPOCH,
            use_tqdm=USE_TQDM_PROGRESS,
        )
        val_loss = run_epoch(
            model_i,
            val_loader,
            optimizer=None,
            desc=f'{name} val e{epoch}',
            max_batches=MAX_VAL_BATCHES_PER_EPOCH,
            use_tqdm=USE_TQDM_PROGRESS,
        )

        history['epoch'].append(epoch)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)

        if np.isfinite(val_loss) and val_loss < best_val:
            best_val = val_loss
            best_state = {k: v.detach().cpu().clone() for k, v in model_i.state_dict().items()}

        print(
            f'[{name}] epoch={epoch:02d} train_loss={train_loss:.4f} '
            f'val_loss={val_loss:.4f} epoch_sec={time.time() - epoch_start:.1f}'
        )

    if len(history['epoch']) == 0:
        print(f'[{name}] no completed epochs within budget; skipping result row.')
        continue

    if best_state is not None:
        model_i.load_state_dict(best_state)

    best_ppl = float(np.exp(best_val)) if np.isfinite(best_val) else np.nan
    history_df_i = pd.DataFrame(history)

    checkpoint_path = None
    if SAVE_MODEL_CHECKPOINTS and (SAVE_ALL_ABLATION_CHECKPOINTS or name == 'baseline'):
        checkpoint_path = MODEL_DIR / f'{training_run_id}_{name}.pt'
        torch.save({
            'model_state_dict': model_i.state_dict(),
            'config': exp,
            'vocab_size': len(itos),
            'seed': SEED,
            'dataset_mode': DATASET_MODE,
            'train_fraction': TRAIN_FRACTION,
            'block_size': BLOCK_SIZE,
            'best_val_loss': best_val,
        }, checkpoint_path)

    run_store[name] = {
        'model': model_i,
        'history_df': history_df_i,
        'best_val_loss': best_val,
        'best_val_perplexity': best_ppl,
        'config': exp,
        'checkpoint_path': str(checkpoint_path) if checkpoint_path is not None else None,
    }

    experiment_rows.append({
        'experiment': name,
        'best_val_loss': best_val,
        'best_val_perplexity': best_ppl,
        'epochs': int(exp['epochs']),
        'd_model': exp['d_model'],
        'n_heads': exp['n_heads'],
        'n_layers': exp['n_layers'],
        'dropout': exp['dropout'],
        'lr': exp['lr'],
        'weight_decay': exp['weight_decay'],
        'checkpoint_path': str(checkpoint_path) if checkpoint_path is not None else None,
        'experiment_wall_sec': round(time.time() - exp_start, 2),
    })

if len(experiment_rows) == 0:
    raise RuntimeError('No experiment completed within current budget/settings. Increase TRAINING_BUDGET_MINUTES or MAX_*_BATCHES_PER_EPOCH.')

experiment_df = pd.DataFrame(experiment_rows).sort_values('best_val_loss', ascending=True).reset_index(drop=True)
display(experiment_df)

final_model_name = experiment_df.iloc[0]['experiment']
model = run_store[final_model_name]['model']
history_df = run_store[final_model_name]['history_df'].copy()
selected_checkpoint_path = run_store[final_model_name]['checkpoint_path']

print({'selected_final_model': final_model_name, 'best_val_loss': float(experiment_df.iloc[0]['best_val_loss']), 'best_val_perplexity': float(experiment_df.iloc[0]['best_val_perplexity'])})

vocab_path = MODEL_DIR / f'{training_run_id}_vocab.json'
with open(vocab_path, 'w', encoding='utf-8') as f:
    json.dump({'itos': itos}, f)
stoi_path = MODEL_DIR / f'{training_run_id}_stoi.json'
with open(stoi_path, 'w', encoding='utf-8') as f:
    json.dump(stoi, f)

if SAVE_MODEL_CHECKPOINTS and selected_checkpoint_path is None:
    selected_checkpoint_path = MODEL_DIR / f'{training_run_id}_{final_model_name}.pt'
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': run_store[final_model_name]['config'],
        'vocab_size': len(itos),
        'seed': SEED,
        'dataset_mode': DATASET_MODE,
        'train_fraction': TRAIN_FRACTION,
        'block_size': BLOCK_SIZE,
        'best_val_loss': run_store[final_model_name]['best_val_loss'],
    }, selected_checkpoint_path)
    selected_checkpoint_path = str(selected_checkpoint_path)

plt.figure(figsize=(10, 5))
for name, artifact in run_store.items():
    h = artifact['history_df']
    plt.plot(h['epoch'], h['val_loss'], marker='o', label=f'{name} val')
plt.title('Validation Loss Curves Across Ablations')
plt.xlabel('Epoch')
plt.ylabel('Cross-Entropy Loss')
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
sns.barplot(data=experiment_df, x='experiment', y='best_val_loss', color='#4C72B0')
plt.title('Best Validation Loss by Experiment')
plt.xlabel('Experiment')
plt.ylabel('Best Validation Loss')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
sns.barplot(data=experiment_df, x='experiment', y='best_val_perplexity', color='#55A868')
plt.title('Best Validation Perplexity by Experiment')
plt.xlabel('Experiment')
plt.ylabel('Perplexity')
plt.tight_layout()
plt.show()

print({'training_run_id': training_run_id, 'selected_checkpoint_path': selected_checkpoint_path, 'vocab_path': str(vocab_path), 'stoi_path': str(stoi_path)})


### Analysis

This comparison reports how architecture and optimization changes affect validation loss and perplexity on the same token stream. The selected final model is the one with the best validation score under a fixed rule, and checkpoint artifacts are saved so future runs can reuse the best model without retraining from raw data.


### Benchmark Visualization

These plots expand the ablation comparison beyond a single ranking by showing trajectory behavior, relative gain over baseline, and the joint relationship between loss and perplexity.

In [ ]:
# Build a long-form history table for trajectory comparisons
history_rows = []
for exp_name, artifact in run_store.items():
    h = artifact['history_df'].copy()
    h['experiment'] = exp_name
    history_rows.append(h)

history_long_df = pd.concat(history_rows, ignore_index=True) if history_rows else pd.DataFrame()

if len(history_long_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(13, 4))
    sns.lineplot(data=history_long_df, x='epoch', y='train_loss', hue='experiment', marker='o', ax=axes[0])
    axes[0].set_title('Train Loss by Epoch and Experiment')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Train Loss')

    sns.lineplot(data=history_long_df, x='epoch', y='val_loss', hue='experiment', marker='o', ax=axes[1])
    axes[1].set_title('Validation Loss by Epoch and Experiment')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Validation Loss')

    plt.tight_layout()
    plt.show()

# Relative improvement over baseline
bench_df = experiment_df.copy()
baseline_loss = float(bench_df.loc[bench_df['experiment'] == 'baseline', 'best_val_loss'].iloc[0]) if (bench_df['experiment'] == 'baseline').any() else float(bench_df['best_val_loss'].iloc[0])
bench_df['relative_improvement_vs_baseline_pct'] = 100.0 * (baseline_loss - bench_df['best_val_loss']) / baseline_loss

display(bench_df[['experiment', 'best_val_loss', 'best_val_perplexity', 'relative_improvement_vs_baseline_pct']])

plt.figure(figsize=(8, 4))
sns.barplot(data=bench_df, x='experiment', y='relative_improvement_vs_baseline_pct', color='#8172B2')
plt.axhline(0.0, color='black', linewidth=1)
plt.title('Relative Validation-Loss Improvement vs Baseline (%)')
plt.xlabel('Experiment')
plt.ylabel('Improvement (%)')
plt.tight_layout()
plt.show()

# Joint metric view
plt.figure(figsize=(7, 5))
ax = sns.scatterplot(data=bench_df, x='best_val_loss', y='best_val_perplexity', hue='experiment', s=120)
for _, row in bench_df.iterrows():
    ax.text(row['best_val_loss'], row['best_val_perplexity'], f" {row['experiment']}", va='center')
plt.title('Ablation Benchmark: Loss vs Perplexity')
plt.xlabel('Best Validation Loss')
plt.ylabel('Best Validation Perplexity')
plt.tight_layout()
plt.show()


## 9) Generate Event Sequences

In [ ]:
def sample_next_token(logits, temperature=1.0, top_k=25):
    logits = logits / max(temperature, 1e-6)
    if top_k is not None and top_k > 0:
        top_vals, top_idx = torch.topk(logits, k=min(top_k, logits.shape[-1]))
        probs = torch.softmax(top_vals, dim=-1)
        choice = torch.multinomial(probs, num_samples=1)
        return top_idx[choice]
    probs = torch.softmax(logits, dim=-1)
    return torch.multinomial(probs, num_samples=1)


def generate_tokens(model, seed_tokens, max_new_tokens=120, temperature=0.9, top_k=25):
    model.eval()
    tokens = seed_tokens.copy()
    x = torch.tensor(tokens, dtype=torch.long, device=DEVICE).unsqueeze(0)

    with torch.no_grad():
        for _ in range(max_new_tokens):
            x_ctx = x[:, -BLOCK_SIZE:]
            logits = model(x_ctx)
            next_logits = logits[0, -1, :]
            next_token = sample_next_token(next_logits, temperature=temperature, top_k=top_k)
            x = torch.cat([x, next_token.view(1, 1)], dim=1)

    return x.squeeze(0).cpu().tolist()


def decode_tokens(token_ids):
    return [itos[t] if 0 <= t < len(itos) else '<UNK>' for t in token_ids]


def split_events(decoded_tokens):
    events = []
    cur = []
    for tok in decoded_tokens:
        if tok == '<EOS>':
            if cur:
                events.append(cur)
                cur = []
        elif tok not in {'<PAD>'}:
            cur.append(tok)
    if cur:
        events.append(cur)
    return events


seed_start = random.randint(0, max(1, len(train_tokens) - BLOCK_SIZE - 1))
seed_seq = train_tokens[seed_start: seed_start + BLOCK_SIZE].tolist()

generated_samples = []
for i in range(NUM_GENERATIONS):
    token_ids = generate_tokens(
        model,
        seed_tokens=seed_seq,
        max_new_tokens=GEN_MAX_NEW_TOKENS,
        temperature=GEN_TEMPERATURE,
        top_k=GEN_TOP_K,
    )
    decoded = decode_tokens(token_ids)
    events = split_events(decoded)

    generated_samples.append({
        'generation_id': f'gen_{i+1:02d}',
        'token_ids': token_ids,
        'decoded_tokens': decoded,
        'events': events,
    })

print('Generated samples:', len(generated_samples))
for s in generated_samples[:2]:
    print('\n', s['generation_id'], 'event_count=', len(s['events']))
    for ev in s['events'][:3]:
        print('  ', ' '.join(ev))

### Analysis

This section provides direct evidence that outputs are generated by your trained model. Review event coherence (token compatibility and repeated structures) and variety across samples before interpreting RCA behavior.

## 10) RCA Narrative Layer Over Generated Events

RCA here is used as model-output interpretation, not as autonomous decision authority. The notebook derives hypothesis text and traceability fields from generated network-flow patterns.

In [ ]:
def tokens_to_event_dict(token_list):
    d = {}
    for tok in token_list:
        if '_' not in tok:
            continue
        prefix = tok.split('_', 1)[0]
        d[prefix] = tok
    return d


def summarize_generated_events(events):
    parsed = [tokens_to_event_dict(ev) for ev in events]
    if not parsed:
        return {
            'generated_summary': 'No coherent events generated.',
            'rca_hypothesis': 'Model output too short for RCA hypothesis.',
            'evidence_refs': [],
            'quality_flags': ['empty_generation'],
            'safety_flags': ['human_review_required'],
            'failure_mode_tags': ['insufficient_content'],
        }

    src_counts = Counter([p.get('SC', 'SC_UNK') for p in parsed])
    dst_counts = Counter([p.get('DC', 'DC_UNK') for p in parsed])
    proto_counts = Counter([p.get('PR', 'PR_UNK') for p in parsed])

    top_src, top_src_n = src_counts.most_common(1)[0]
    top_dst, top_dst_n = dst_counts.most_common(1)[0]
    top_proto, top_proto_n = proto_counts.most_common(1)[0]

    total = len(parsed)
    unique_dst = len(dst_counts)
    dst_spread_ratio = unique_dst / max(total, 1)
    proto_diversity = len(proto_counts)

    quality_flags = []
    failure_tags = []

    if dst_spread_ratio > 0.65:
        rca = 'Possible lateral movement or destination fan-out pattern in generated flow behavior.'
        quality_flags.append('high_destination_spread')
    elif proto_diversity > 3:
        rca = 'Mixed protocol behavior may indicate scanning or multiplexed service activity.'
        quality_flags.append('high_protocol_diversity')
    else:
        rca = 'Predominantly repetitive network-flow behavior with limited protocol variability.'

    if total < 6:
        failure_tags.append('short_sequence')
    if top_proto == 'PR_UNK':
        failure_tags.append('low_semantic_specificity')

    summary = (
        f'Generated {total} events. Dominant source token {top_src} ({top_src_n} events), '
        f'dominant destination token {top_dst} ({top_dst_n} events), '
        f'primary protocol token {top_proto} ({top_proto_n} events), '
        f'destination spread ratio {dst_spread_ratio:.3f}, protocol diversity {proto_diversity}.'
    )

    evidence_refs = [
        {'type': 'generated_event_index', 'value': int(i)} for i in range(min(5, total))
    ]

    safety_flags = ['generated_not_ground_truth', 'human_validation_required']

    return {
        'generated_summary': summary,
        'rca_hypothesis': rca,
        'evidence_refs': evidence_refs,
        'quality_flags': quality_flags,
        'safety_flags': safety_flags,
        'failure_mode_tags': failure_tags,
    }


rca_rows = []
for sample in generated_samples:
    interp = summarize_generated_events(sample['events'])
    row = {
        'generation_id': sample['generation_id'],
        'event_count': len(sample['events']),
        **interp,
    }
    rca_rows.append(row)

rca_df = pd.DataFrame(rca_rows)
display(rca_df[['generation_id', 'event_count', 'generated_summary', 'rca_hypothesis', 'quality_flags', 'failure_mode_tags']])


### Analysis

RCA summaries are now mode-aware: network runs use `SC/DC/PR` tokens (source, destination, protocol), while auth runs use `SU/ST/AT` tokens. This avoids schema mismatch and produces interpretations grounded in the generated event type rather than generic placeholders.

## 11) Graph Views of Generated Interactions and RCA Links

This section visualizes generated structure as graphs: an interaction graph from generated source/destination tokens and an RCA relation graph linking generations to hypotheses and quality flags.

In [ ]:
# Graph 1: generated interaction graph (aggregated across samples)
interaction_edges = Counter()
for sample in generated_samples:
    for ev in sample['events']:
        d = tokens_to_event_dict(ev)
        src = d.get('SC', 'SC_UNK')
        dst = d.get('DC', 'DC_UNK')
        interaction_edges[(src, dst)] += 1

# keep top edges for readability
top_k_edges = 40
top_edges = interaction_edges.most_common(top_k_edges)

G = nx.DiGraph()
for (src, dst), w in top_edges:
    G.add_edge(src, dst, weight=w)

plt.figure(figsize=(12, 8))
if G.number_of_nodes() > 0:
    pos = nx.spring_layout(G, seed=SEED, k=0.7)
    edge_w = [G[u][v]['weight'] for u, v in G.edges()]
    max_w = max(edge_w) if edge_w else 1
    edge_w_norm = [1 + 4 * (w / max_w) for w in edge_w]

    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='#4C72B0', alpha=0.85)
    nx.draw_networkx_edges(G, pos, width=edge_w_norm, alpha=0.45, edge_color='#55A868', arrows=True, arrowsize=14)
    nx.draw_networkx_labels(G, pos, font_size=8)
    plt.title('Generated Interaction Graph (Top Weighted Edges)')
    plt.axis('off')
else:
    plt.text(0.5, 0.5, 'No graph edges available', ha='center', va='center')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Graph 2: RCA relation graph (generation -> hypothesis/flags)
R = nx.DiGraph()
for row in rca_rows:
    gen = row['generation_id']
    hyp = f"HYP: {row['rca_hypothesis'][:80]}"
    R.add_edge(gen, hyp, rel='hypothesis')

    qf = row.get('quality_flags', []) or []
    if len(qf) == 0:
        R.add_edge(gen, 'quality:none', rel='quality')
    else:
        for q in qf:
            R.add_edge(gen, f'quality:{q}', rel='quality')

    ff = row.get('failure_mode_tags', []) or []
    if len(ff) == 0:
        R.add_edge(gen, 'failure:none', rel='failure')
    else:
        for ftag in ff:
            R.add_edge(gen, f'failure:{ftag}', rel='failure')

plt.figure(figsize=(13, 8))
if R.number_of_nodes() > 0:
    pos = nx.spring_layout(R, seed=SEED, k=0.9)
    node_colors = []
    for n in R.nodes():
        if str(n).startswith('gen_'):
            node_colors.append('#4C72B0')
        elif str(n).startswith('HYP:'):
            node_colors.append('#C44E52')
        elif str(n).startswith('quality:'):
            node_colors.append('#55A868')
        else:
            node_colors.append('#8172B2')

    nx.draw_networkx_nodes(R, pos, node_size=560, node_color=node_colors, alpha=0.9)
    nx.draw_networkx_edges(R, pos, alpha=0.35, arrows=True, arrowsize=12)
    nx.draw_networkx_labels(R, pos, font_size=8)
    plt.title('RCA Relation Graph (Generation -> Hypothesis/Flags)')
    plt.axis('off')
else:
    plt.text(0.5, 0.5, 'No RCA graph edges available', ha='center', va='center')
    plt.axis('off')
plt.tight_layout()
plt.show()


### Analysis

The interaction graph highlights repeated and high-weight generated communication paths, while the RCA relation graph shows how each generation maps to hypotheses and quality/failure tags. Together, these views make generation behavior auditable and easier to discuss in system-level RCA terms.

## 12) Generation Quality Diagnostics

In [ ]:
def distinct_n(tokens, n=1):
    if len(tokens) < n:
        return 0.0
    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    return len(set(ngrams)) / max(len(ngrams), 1)


def repetition_ratio(tokens):
    if not tokens:
        return 0.0
    return 1.0 - (len(set(tokens)) / len(tokens))


quality_rows = []
for s in generated_samples:
    toks = [t for t in s['decoded_tokens'] if t not in {'<PAD>'}]
    d1 = distinct_n(toks, n=1)
    d2 = distinct_n(toks, n=2)
    rep = repetition_ratio(toks)
    eos_count = sum(1 for t in toks if t == '<EOS>')

    quality_rows.append({
        'generation_id': s['generation_id'],
        'token_count': len(toks),
        'distinct_1': d1,
        'distinct_2': d2,
        'repetition_ratio': rep,
        'eos_count': eos_count,
    })

quality_df = pd.DataFrame(quality_rows)
display(quality_df)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.barplot(data=quality_df, x='generation_id', y='distinct_2', ax=axes[0], color='#4C72B0')
axes[0].set_title('Distinct-2 by Generated Sample')
axes[0].set_xlabel('Generation ID')
axes[0].set_ylabel('Distinct-2')

sns.barplot(data=quality_df, x='generation_id', y='repetition_ratio', ax=axes[1], color='#DD8452')
axes[1].set_title('Repetition Ratio by Generated Sample')
axes[1].set_xlabel('Generation ID')
axes[1].set_ylabel('Repetition Ratio')

plt.tight_layout()
plt.show()

### Analysis

Use these diagnostics to identify weak generations (for example, very high repetition or very low distinct-2). Combined with RCA narrative inspection, this gives a practical quality lens without claiming that generated outputs are operationally accurate.

## 13) Export Structured Output Contract (for later P7 integration)

This export is generated in P5 only and does not create a dependency on P6.

In [ ]:
artifact_rows = []
run_id = training_run_id if 'training_run_id' in globals() else f'p5_lanl_transformer_seed{SEED}_{datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")}'

for sample, interp in zip(generated_samples, rca_rows):
    artifact_rows.append({
        'incident_id': f'GEN_INC_{sample["generation_id"]}',
        'generation_id': sample['generation_id'],
        'generated_event_sequence': ' | '.join(' '.join(ev) for ev in sample['events'][:20]),
        'generated_summary': interp['generated_summary'],
        'rca_hypothesis': interp['rca_hypothesis'],
        'evidence_refs': interp['evidence_refs'],
        'quality_flags': interp['quality_flags'],
        'safety_flags': interp['safety_flags'],
        'failure_mode_tags': interp['failure_mode_tags'],
        'model_version': 'token_transformer_lm_v1',
        'selected_model_name': final_model_name if 'final_model_name' in globals() else 'baseline',
        'selected_model_checkpoint': selected_checkpoint_path if 'selected_checkpoint_path' in globals() else None,
        'seed': SEED,
        'run_id': run_id,
        'timestamp_utc': datetime.now(timezone.utc).isoformat(),
    })

artifact_df = pd.DataFrame(artifact_rows)
display(artifact_df.head())

jsonl_path = PROCESSED_DIR / 'p5_generated_rca_artifacts.jsonl'
csv_path = PROCESSED_DIR / 'p5_generated_rca_artifacts.csv'

artifact_df.to_csv(csv_path, index=False)
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for row in artifact_rows:
        f.write(json.dumps(row) + '\n')

print({'csv_path': str(csv_path), 'jsonl_path': str(jsonl_path), 'rows': len(artifact_rows)})


## 14) Ethical Considerations and Responsible Use

This project treats generated outputs as synthetic hypotheses, not incident facts. Main risks include fabricated causal narratives, bias from dominant behavioral patterns in the training stream, and misuse if generated recommendations are actioned without human validation.

A critical risk in this context is spoofing or injection: adversarially crafted or poisoned event sequences could drive the generative layer to produce plausible but misleading narratives, which can contaminate reasoning and downstream decision-support systems if consumed without verification.

Mitigations used here:
- explicit `generated_not_ground_truth` safety labeling,
- mandatory human-review flag,
- failure-mode tagging for low-specificity or short outputs,
- structured evidence references for traceability,
- strict separation between generation and operational action decisions,
- recommendation to validate generated RCA hypotheses against independent telemetry before escalation.


## 15) V&V Checklist

In [ ]:
vnv = {
    'dataset_loaded': isinstance(raw_df, pd.DataFrame) and len(raw_df) >= 2000,
    'processed_event_cache_available': PROCESSED_EVENTS_PATH.exists(),
    'model_defined': isinstance(model, nn.Module),
    'ablation_table_available': isinstance(experiment_df, pd.DataFrame) and len(experiment_df) >= 1,
    'selected_model_named': isinstance(final_model_name, str) and len(final_model_name) > 0,
    'checkpoint_saved': bool('selected_checkpoint_path' in globals() and selected_checkpoint_path),
    'training_history_available': isinstance(history_df, pd.DataFrame) and len(history_df) >= 1,
    'generated_samples_present': len(generated_samples) >= 3,
    'qualitative_rca_present': isinstance(rca_df, pd.DataFrame) and len(rca_df) >= 3,
    'quality_diagnostics_present': isinstance(quality_df, pd.DataFrame) and len(quality_df) >= 3,
    'export_artifacts_written': csv_path.exists() and jsonl_path.exists(),
}

print('V&V status:')
print(vnv)


## 16) Short Notebook Summary

This notebook implemented a Transformer-based generative model for LANL event-sequence data and produced multiple generated samples from trained model weights. To keep the workflow portable, the ingestion pipeline supports deterministic size-capped subsets and writes processed caches so reruns do not require re-reading multi-GB raw files. Controlled ablations compared architecture and optimization variants, and the final model was selected using a fixed validation-loss criterion with checkpoint export. Graph-based views of generated interactions and RCA links were added to improve interpretability. Generated outputs were interpreted using an RCA-oriented narrative layer that emits hypotheses, evidence references, and safety flags. The main challenge is balancing sequence diversity with semantic specificity when entity identifiers are bucketed for tractability. A key limitation is that generated narratives are hypothesis artifacts and must be validated against real telemetry evidence before operational use.


## 17) Report Notes

For `Generative_AI_Analysis_Report.pdf`, include:
- why Transformer-based sequence generation is appropriate for LANL-style event streams,
- how deterministic size-capped subset creation improves reproducibility and reviewer portability,
- EDA-backed rationale (distribution shape, covariance, source balance) for tokenization and training design,
- ablation comparison results and final-model selection rule,
- graph-based interpretation of generated interactions and RCA relation links,
- specific generated examples with strengths and failure cases,
- explicit ethical risks (bias, misuse, fabricated evidence, spoofing/injection into reasoning workflows, creative ownership concerns),
- limitations and future improvements,
- a short P7 integration note describing exported artifact schema, checkpoint assets, and system boundaries.
